In [ ]:
import wandb
import mitsuba as mi
mi.set_variant("cuda_ad_rgb")

import numpy as np
import matplotlib.pyplot as plt

import import_ipynb
import importlib

import testing_scene as ts
importlib.reload(ts)

import vapl_utils as utils
importlib.reload(utils)

sweep_config = {
    "method": "bayes",
    "metric": {
        "name": "loss",
        "goal": "minimize"
    },
    "parameters": {
        "gaussian_mean_encoding": {
            "values": ["raw", "eps-norm", "min-max-norm"]
        },
        "gaussian_variance_encoding": {
            "values": ["exp", "sigmoid", "softplus"]
        },
        "vmf_sharpness_encoding": {
            "values": ["exp", "relu", "sigmoid", "softplus"]
        },
        "vmf_axis_encoding": {
            "values": ["raw", "normalize", "spherical", "spherical-norm"]
        },
        "vmf_amplitude_encoding": {
            "values": ["relu", "softplus", "exp"]
        },
        "learning_rate": {
            "values":[0.001, 0.0001]
        },
        "epoch": {"values": [200]}
    },
}

In [ ]:
def weighted_loss(real, predicted, weight):
    eps = 0.01
    mse = (real - predicted) ** 2
    norm_factor = (weight * (predicted ** 2).detach() + eps)
    return (mse / (norm_factor + eps)).mean()

loss_function = utils.Loss(weighted_loss)

def main():
    wandb.init("vapls-parameters-encodings-search")
    field = utils.vapl_grid(ts.scene.bbox().min, ts.scene.bbox().max, 1, 4, 8, wandb.config).cuda()
    rhs_integrator = utils.RHSIntegrator(field, loss_function)

    for epoch in range(wandb.config.epoch):
        rhs_image = mi.render(ts.scene, spp=1, integrator=rhs_integrator)
        wandb.log({"loss": rhs_integrator.losses[-1].item(), "epoch": epoch})

wandb.login()
sweep_id = wandb.sweep(sweep_config, project="vapl-sweep-test")
wandb.agent(sweep_id, main)